In [ ]:
import requests
from bs4 import BeautifulSoup
import os
import json
import pprint as pp

# Create a directory to saving files
file_path = r'./manpower104'
if not os.path.exists(file_path): os.mkdir(file_path)  
    
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.111 Safari/537.36'} 
keyword = '後端工程師'

# synonym dictionary
skill_dict = {}
skill_path = r'./manpower104/skill/skilldict.txt'
with open(skill_path,'r',encoding='utf-8-sig') as s:
    s_str = s.read().split('\n')
for each_row in s_str:
    skill_dict[each_row.split(',')[0]] = 0
# print(skill_dict)

# with open(synonym_path, 'r', encoding='utf-8') as syn:
#     syn_str = syn.read().split('\n')
# for each_row in syn_str:
#     synonym_dict[each_row.split(',')[0]] = [item for item in each_row.split(',')]


url_start = "https://www.104.com.tw/jobs/search/?ro=0&isnew=30&keyword=%s"%(keyword)
res_start = requests.get(url_start, headers=headers)

# get total pages of 104
pages=int(res_start.text.split('"totalPage":')[1].split(',"totalCount"')[0])

# how many pages need scrap.
for page in range(1):
    res_page=requests.get(url_start+"&page=%s"%(page))
    soup_page = BeautifulSoup(res_page.text, 'html.parser')
    article_page_list = soup_page.select('h2.b-tit a')

# create title && title url
    for article in article_page_list:
        article_title = article.text
        url_article = article['href'].replace('//', 'https://')
        url_article_json = 'https://www.104.com.tw/job/ajax/content/' + url_article.split('/')[-1]
        
        # set new header of referer included.
        headers_j = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.111 Safari/537.36',
                  'referer':url_article.split('?')[0]
                    } 
        res_article = requests.get(url_article_json, headers=headers_j) 
        job_content = json.loads(res_article.text)

#         pp.pprint(job_content)
#         print(url_article)

# def information
        job_name = (job_content['data']['header']['jobName'])
        job_company = (job_content['data']['header']['custName'])
        job_location = (job_content['data']['jobDetail']['addressRegion'])
        job_salary = (job_content['data']['jobDetail']['salary'])
        job_years = (job_content['data']['condition']['workExp'])
        job_skill = (job_content['data']['jobDetail']['jobDescription']+job_content['data']['condition']['other'])
#         print(job_skill)
        
# sum information       
        job_allinfo = ''
        job_allinfo += '工作名稱: %s\n' %(job_name)
        job_allinfo += '公司名稱: %s\n'%(job_company)
        job_allinfo += 'URL: %s\n'%(url_article)
        job_allinfo += '公司地點: %s\n'%(job_location)
        job_allinfo += '工作薪水: %s\n'%(job_salary)
        job_allinfo += '工作經歷: %s\n'%(job_years)
        
#         print(type(job_skill))
#         print(job_skill)
#         print(job_allinfo)
#         print('='*50)

# count skill       
        wordlist = jieba.cut(str(job_skill).lower())
        jieba.load_userdict('./manpower104/skill/skilldict.txt')
        cutlist = [i for i in wordlist]
        for w in skill_dict:
            if w in cutlist:
                job_allinfo += '%s: 1\n'%(w)
            else:
                job_allinfo += '%s: 0\n'%(w)  
        print(job_allinfo)
        print('='*50)
    
# save file
        try:
            with open( '%s/%s.txt' %(file_path, job_company+'_'+article_title), 'w', encoding='utf-8-sig') as f:
                f.write(job_allinfo)

        except FileNotFoundError as e:
            article_title = article_title.replace('/','&&')
            
            print(len(article_page_list))
            print(article_title)
            print(url_article)
            print(e.args)
            with open( '%s/%s.txt' %(file_path, job_company+'_'+article_title), 'w', encoding='utf-8-sig') as f:
                f.write(job_allinfo)
            
        except OSError as e:
            print('==========')
            print(url_article)
            print(e.args)
            print('==========')


In [ ]:
#create form table
import pandas as pd
import glob

filelist = glob.glob('./manpower104/*.txt')

rowData = []

for filename in filelist:
    with open(filename, 'r', encoding='utf-8-sig') as f:
        tmplist = f.read().split('\n')[0:-1]
        rowData.append(tmplist)       
        print(rowData)

In [ ]:
# create column
columns = [r.split(': ')[0] for r in rowData[0]]

# create table
df = pd.DataFrame(data=rowData, columns=columns)
print(df)

In [ ]:
# remove title words
for c in df:
    df[c] = df[c].apply(lambda x: x.split(': ')[-1])
print(df)

In [ ]:
# save cvs.file
df.to_csv('./manpower104/manpower_table.csv') 
print('done!')